In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

input="../input/"
train=pd.read_csv(input+"train_1.csv")
train.head(5)

def find_language(url):
    res=re.search('[a-z][a-z].wikipedia.org',url)
    if res:
        return res[0][0:2]
    return 'na'
train['lang']=train.Page.map(find_language)
train['lang'].head(5)

In [ ]:
lang_uni=pd.unique(train['lang'])
lang_sets={}
for i in range(len(lang_uni)):
    lang_sets[str(lang_uni[i])]=train[train.lang==lang_uni[i]].iloc[:,0:-1]
    
sums={}

for key in lang_uni:
    sums[key]=pd.DataFrame(lang_sets[key].iloc[:,1:].sum(axis=0)/lang_sets[key].shape[0]).reset_index(drop=True)
    sums[key].columns=['Visits']

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

proc_train=train.fillna(0)

final_pred={}
for key in sums:
    temp=np.array(sums[key])
    X=temp[0:549]
    Y=temp[1:550]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)
    
    sc = MinMaxScaler()
    X_train = np.reshape(X_train,(-1,1))
    Y_train = np.reshape(Y_train,(-1,1))
    X_train = sc.fit_transform(X_train)
    Y_train = sc.fit_transform(Y_train)
    
    X_train = np.reshape(X_train, (384,1,1))
    #Initialise RNN
    regressor = Sequential()
    
    #Add LSTM layer
    regressor.add(LSTM(units = 8, activation = 'relu', input_shape = (None, 1)))
    
    #Add output layer
    regressor.add(Dense(units = 1))

    # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    # Fitting the RNN to the Training set
    regressor.fit(X_train, Y_train, batch_size = 10, epochs = 100, verbose = 0)
    
    # Getting the predicted Web View
    inputs = X_test
    inputs = np.reshape(inputs,(-1,1))
    inputs = sc.transform(inputs)
    inputs = np.reshape(inputs, (165, 1, 1))
    Y_pred = regressor.predict(inputs)
    Y_pred = sc.inverse_transform(Y_pred)
    
    list_test=X
    pred=[]
    for i in range(60):
        print(i)
        inputs =list_test[i+0:i+549]
        inputs = sc.transform(inputs)
        inputs = np.reshape(inputs, (549, 1, 1))
        Y_pred = regressor.predict(inputs)
        temp=Y_pred[:-1]
        a=temp.tolist()
        b=a[-1][-1]
        pred.append(b)
        list_test = np.concatenate([list_test, a])

    final_pred[key]=pred
    

In [ ]:
import datetime 
date1 = '2011-01-01'
date2 = '2011-03-01'
mydates = pd.date_range(date1, date2).tolist()
dates=[]
for temp in mydates:
    temp1 = str(temp)[0:10]
    dates.append(temp1)
print(dates[0])

In [ ]:
output=train[['Page','lang']]

for i in range(len(dates)):
    output[str(dates[i])]=final_pred[output['lang'][i]][i]
    
    

In [ ]:
output.tail(5)